2020/10/24
30秒くらいまで。ぜんぜんだめだ。これを数値解析の目標にして勉強しよう

問題設定は(http://articles.adsabs.harvard.edu/cgi-bin/nph-iarticle_query?1967AJ.....72..876S&amp;data_type=PDF_HIGH&amp;whole_paper=YES&amp;type=PRINTER&amp;filetype=.pdf)

In [1]:
import sympy as sy
import sympy.physics.mechanics as me

In [13]:
# イレギュラーだけど、論文に倣って、名前は3,4,5にする
N = me.ReferenceFrame('N')

Po3 = me.Point("Po3")
m3 = sy.Symbol("m3")
Pa3 = me.Particle("Pa3", Po3, m3)

Po4 = me.Point("Po4")
m4 = sy.Symbol("m4")
Pa4 = me.Particle("Pa4", Po4, m4)

Po5 = me.Point("Po5")
m5 = sy.Symbol("m5")
Pa5 = me.Particle("Pa5", Po5, m5)

t = sy.Symbol('t')
x3, y3 = me.dynamicsymbols("x3 y3")
vx3 = sy.diff(x3, t)
vy3 = sy.diff(y3, t)
Po3.set_vel(N, vx3*N.x+vy3*N.y)

x4, y4 = me.dynamicsymbols("x4 y4")
vx4 = sy.diff(x4, t)
vy4 = sy.diff(y4, t)
Po4.set_vel(N, vx4*N.x+vy4*N.y)

x5, y5 = me.dynamicsymbols("x5 y5")
vx5 = sy.diff(x5, t)
vy5 = sy.diff(y5, t)
Po5.set_vel(N, vx5*N.x+vy5*N.y)

In [14]:
L = me.Lagrangian(N, Pa3, Pa4, Pa5)

In [19]:
qs = sy.Matrix([x3,y3,x4,y4,x5,y5])

G = sy.Symbol('G')
r34_sq = (x3-x4)**2 + (y3-y4)**2
r45_sq = (x4-x5)**2 + (y4-y5)**2
r53_sq = (x5-x3)**2 + (y5-y3)**2
angle34 = sy.atan( (y3-y4)/(x3-x4) )
angle45 = sy.atan( (y4-y5)/(x4-x5) )
angle53 = sy.atan( (y5-y3)/(x5-x3) )
forcelist = [(Po3, G*m3*m4/r34_sq*(-sy.cos(angle34)*N.x -sy.sin(angle34)*N.y) + G*m5*m3/r53_sq*(sy.cos(angle53)*N.x +sy.sin(angle53)*N.y) ),\
            (Po4, G*m4*m5/r45_sq*(-sy.cos(angle45)*N.x -sy.sin(angle45)*N.y) + G*m3*m4/r34_sq*(sy.cos(angle34)*N.x +sy.sin(angle34)*N.y) ),\
            (Po5, G*m5*m3/r53_sq*(-sy.cos(angle53)*N.x -sy.sin(angle53)*N.y) + G*m4*m5/r45_sq*(sy.cos(angle45)*N.x +sy.sin(angle45)*N.y) )]

In [22]:
LM = me.LagrangesMethod(L, qs=qs, forcelist=forcelist, frame=N)

In [25]:
eom = LM.form_lagranges_equations()
eom

Matrix([
[                                                                  G*m3*m4/(sqrt(1 + (y3(t) - y4(t))**2/(x3(t) - x4(t))**2)*((x3(t) - x4(t))**2 + (y3(t) - y4(t))**2)) - G*m3*m5/(sqrt(1 + (-y3(t) + y5(t))**2/(-x3(t) + x5(t))**2)*((-x3(t) + x5(t))**2 + (-y3(t) + y5(t))**2)) + m3*Derivative(x3(t), (t, 2))],
[G*m3*m4*(y3(t) - y4(t))/(sqrt(1 + (y3(t) - y4(t))**2/(x3(t) - x4(t))**2)*((x3(t) - x4(t))**2 + (y3(t) - y4(t))**2)*(x3(t) - x4(t))) - G*m3*m5*(-y3(t) + y5(t))/(sqrt(1 + (-y3(t) + y5(t))**2/(-x3(t) + x5(t))**2)*((-x3(t) + x5(t))**2 + (-y3(t) + y5(t))**2)*(-x3(t) + x5(t))) + m3*Derivative(y3(t), (t, 2))],
[                                                                     -G*m3*m4/(sqrt(1 + (y3(t) - y4(t))**2/(x3(t) - x4(t))**2)*((x3(t) - x4(t))**2 + (y3(t) - y4(t))**2)) + G*m4*m5/(sqrt(1 + (y4(t) - y5(t))**2/(x4(t) - x5(t))**2)*((x4(t) - x5(t))**2 + (y4(t) - y5(t))**2)) + m4*Derivative(x4(t), (t, 2))],
[     -G*m3*m4*(y3(t) - y4(t))/(sqrt(1 + (y3(t) - y4(t))**2/(x3(t) - x4(t

In [26]:
f = LM.rhs()
f

Matrix([
[                                                                                                                                                                                                                                                                 Derivative(x3(t), t)],
[                                                                                                                                                                                                                                                                 Derivative(y3(t), t)],
[                                                                                                                                                                                                                                                                 Derivative(x4(t), t)],
[                                                                                                                                                   

In [27]:
f.shape

(12, 1)

In [32]:
f[8]

(G*m3*m4/(sqrt(1 + (y3(t) - y4(t))**2/(x3(t) - x4(t))**2)*((x3(t) - x4(t))**2 + (y3(t) - y4(t))**2)) - G*m4*m5/(sqrt(1 + (y4(t) - y5(t))**2/(x4(t) - x5(t))**2)*((x4(t) - x5(t))**2 + (y4(t) - y5(t))**2)))/m4

In [2]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt

In [22]:
G = 1 #6.67e-11
M3 = 3
M4 = 4
M5 = 5

def func(state, t):
    dXdt = np.zeros_like(state)
    
    dXdt[0] = state[6]
    dXdt[1] = state[7]
    dXdt[2] = state[8]
    dXdt[3] = state[9]
    dXdt[4] = state[10]
    dXdt[5] = state[11]
    
    r34_sq = (state[0]-state[2])**2 + (state[1]-state[3])**2
    r45_sq = (state[2]-state[4])**2 + (state[3]-state[5])**2
    r53_sq = (state[4]-state[0])**2 + (state[5]-state[1])**2
    
    dXdt[6] = - ( G*M4*(state[0]-state[2]) ) / ( np.sqrt(r34_sq)*r34_sq ) + ( G*M5*(state[4]-state[0]) ) / ( np.sqrt(r53_sq)*r53_sq )
    dXdt[7] = - ( G*M4*(state[1]-state[3]) ) / ( np.sqrt(r34_sq)*r34_sq ) + ( G*M5*(state[5]-state[1]) ) / ( np.sqrt(r53_sq)*r53_sq )
    dXdt[8] = - ( G*M5*(state[2]-state[4]) ) / ( np.sqrt(r45_sq)*r45_sq ) + ( G*M3*(state[0]-state[2]) ) / ( np.sqrt(r34_sq)*r34_sq )
    dXdt[9] = - ( G*M5*(state[3]-state[5]) ) / ( np.sqrt(r45_sq)*r45_sq ) + ( G*M3*(state[1]-state[3]) ) / ( np.sqrt(r34_sq)*r34_sq )
    dXdt[10] = - ( G*M3*(state[4]-state[0]) ) / ( np.sqrt(r53_sq)*r53_sq ) + ( G*M4*(state[2]-state[4]) ) / ( np.sqrt(r45_sq)*r45_sq )
    dXdt[11] = - ( G*M3*(state[5]-state[1]) ) / ( np.sqrt(r53_sq)*r53_sq ) + ( G*M4*(state[3]-state[5]) ) / ( np.sqrt(r45_sq)*r45_sq )
    
    return dXdt

## 初期値
x3 = 1
y3 = 3
x4 = -2
y4 = -1
x5 = 1
y5 = -1
vx3 = 0
vy3 = 0
vx4 = 0
vy4 = 0
vx5 = 0
vy5 = 0

state = [x3,y3, x4,y4, x5,y5, vx3,vy3, vx4,vy4, vx5,vy5]

dt = 0.01
t = np.arange(0,70, dt)

sol = odeint(func, state, t)

print("t x3 y3 x4 y4 x5 y5")

for i in range(7):
    col = i*int(10/dt)
    print("%.1f %.5f %.5f %.5f %.5f %.5f %.5f" % (t[col],sol[col,0],sol[col,1],sol[col,2],sol[col,3],sol[col,4],sol[col,5]))

t x3 y3 x4 y4 x5 y5
0.0 1.00000 3.00000 -2.00000 -1.00000 1.00000 -1.00000
10.0 0.77853 0.14154 -2.02508 0.09712 1.15295 -0.16263
20.0 3.00375 0.51228 -1.38976 -0.47071 -0.69044 0.06920
30.0 0.85695 2.28816 -0.88144 -0.86664 0.19098 -0.67958
40.0 -0.59980 1.88660 0.15233 -2.32548 0.23801 0.72843
50.0 -2.50711 1.97178 0.71240 -1.31815 0.93434 -0.12855
60.0 -1.26343 0.24593 0.24101 -0.88699 0.56525 0.56204


論文とかの値は無次元化してるらしい。つまり、G=1

10ごとのtに対する三体の座標（パソコンで見る天体の動き、p183より）
この数値に合えば正しく計算してることになります。

|  t   |     x3    |     y3  |       x4  |       y4   |      x5    |     y5|
|--: |---|---|---|---|---|---|
| 0.0 |   1.00000 |   3.00000 |  -2.00000 |  -1.00000 |   1.00000 |  -1.00000|
|10.0 |   0.77848 |   0.14139 |  -2.02509 |   0.09722 |   1.15299 |  -0.16261|
|20.0 |   3.00429 |   0.51193 |  -1.38863 |  -0.47048 |  -0.69167 |   0.06923|
|30.0 |   0.85634 |   2.28709 |  -0.87798 |  -0.86596 |   0.18858 |  -0.67949|
|40.0 |  -0.62200 |   1.85832 |   0.17354 |  -2.36841 |   0.23437 |   0.77974|
|50.0 |  -2.70146 |  -3.79722 |   1.50594 |   0.96081 |   0.41513 |   1.50968|
|60.0 |   0.74381 |   1.93995 |   0.26401 |  -0.73163 |  -0.65749 |  -0.57867|